In [14]:
import pandas as pd
import numpy as np
from pathlib import Path
from thefuzz.process import extractOne
from thefuzz.fuzz import partial_token_sort_ratio
from tqdm.auto import tqdm

In [15]:
abbreviations = {
    'Player': 'player',
    'Team': 'teamcurr',
    'Team within selected timeframe' : 'team',
    'Position': 'pos',
    'Age': 'age',
    'Market value': 'marketval',
    'Contract expires': 'ctcexp',
    'Matches played': 'matchesplayed',
    'Minutes played': 'minsplayed',
    'Goals': 'goals',
    'xG': 'xg',
    'Assists': 'assists',
    'xA': 'xa',
    'Duels per 90': 'duels90',
    'Duels won, %': 'duelswonpct',
    'Birth country':'birthcountry',
    'Passport country':'passpcountry',
    'Foot': 'foot',
    'Height': 'height',
    'Weight': 'weight',
    'On loan': 'loanflg',
    'Successful defensive actions per 90': 'succssdefact90',
    'Defensive duels per 90':'defduels90',
    'Defensive duels won, %': 'defduelswpct',
    'Aerial duels per 90': 'aerduels90',
    'Aerial duels won, %': 'aerduelspct',
    'Sliding tackles per 90': 'sltackles90',
    'PAdj Sliding tackles': 'padjsltackles',
    'Shots blocked per 90': 'shotblocks90',
    'Interceptions per 90': 'intrcpt90',
    'PAdj Interceptions': 'padjintrcpt',
    'Fouls per 90': 'fouls90',
    'Yellow cards': 'ycards',
    'Yellow cards per 90': 'ycards90',
    'Red cards': 'rcards',
    'Red cards per 90': 'rcards90',
    'Successful attacking actions per 90': 'sattact90',
    'Goals per 90': 'goals90',
    'Non-penalty goals': 'npgoals',
    'Non-penalty goals per 90': 'npgoals90',
    'xG per 90': 'xg90',
    'Head goals': 'hdrgoals',
    'Head goals per 90': 'hdrgoals90',
    'Shots': 'shots',
    'Shots per 90': 'shots90',
    'Shots on target, %': 'ontrgtpct',
    'Goal conversion, %': 'goalcnvrsnpct',
    'Assists per 90': 'assists90',
    'Crosses per 90': 'crosses90',
    'Accurate crosses, %': 'acccrossespct',
    'Crosses from left flank per 90': 'crossesleft90',
    'Accurate crosses from left flank, %': 'acccrossesleftpct',
    'Crosses from right flank per 90': 'crossesright90',
    'Accurate crosses from right flank, %': 'acccrossesrightpct',
    'Crosses to goalie box per 90': 'crosses6yard90',
    'Dribbles per 90': 'dribbles90',
    'Successful dribbles, %': 'succssdribblespct',
    'Offensive duels per 90': 'offduels90',
    'Offensive duels won, %': 'offduelspct',
    'Touches in box per 90': 'touchesbox90',
    'Progressive runs per 90': 'prgruns90',
    'Accelerations per 90': 'acclr90',
    'Received passes per 90': 'rcvdpasses90',
    'Received long passes per 90': 'rcvdlongpasses90',
    'Fouls suffered per 90': 'foulsa90',
    'Passes per 90': 'passes90',
    'Accurate passes, %': 'accpassespct',
    'Forward passes per 90': 'fwdpasses90',
    'Accurate forward passes, %': 'accfwdpassespct',
    'Back passes per 90': 'bpasses90',
    'Accurate back passes, %': 'accbpassespct',
    'Lateral passes per 90': 'latpasses90',
    'Accurate lateral passes, %': 'acclatpassespct',
    'Short / medium passes per 90': 'shortmedpasses90',
    'Accurate short / medium passes, %': 'accshortmedpassespct',
    'Long passes per 90': 'longpasses90',
    'Accurate long passes, %': 'acclongpassesct',
    'Average pass length, m': 'avgpasslen',
    'Average long pass length, m': 'avglongpasslen',
    'xA per 90': 'xa90',
    'Shot assists per 90': 'shotassist90',
    'Second assists per 90': 'secassist90',
    'Third assists per 90': 'thirdassist90',
    'Smart passes per 90': 'smartpasses90',
    'Accurate smart passes, %': 'accsmartpassespct',
    'Key passes per 90': 'kp90',
    'Passes to final third per 90': 'final3rdpasses90',
    'Accurate passes to final third, %': 'accfinal3rdpassespct',
    'Passes to penalty area per 90': 'penareapasses90',
    'Accurate passes to penalty area, %': 'accpenareapassespct',
    'Through passes per 90': 'thrpasses90',
    'Accurate through passes, %': 'accthrpassespct',
    'Deep completions per 90': 'deepcomp90',
    'Deep completed crosses per 90': 'deepcompcrosses90',
    'Progressive passes per 90': 'prpasses90',
    'Accurate progressive passes, %': 'accprpassespct',
    'Conceded goals': 'ga',
    'Conceded goals per 90': 'ga90',
    'Shots against': 'shotsa',
    'Shots against per 90': 'shotsa90',
    'Clean sheets': 'cs',
    'Save rate, %': 'saveratepct',
    'xG against': 'xga',
    'xG against per 90': 'xga90',
    'Prevented goals': 'prevgoals',
    'Prevented goals per 90': 'prevgoals90',
    'Back passes received as GK per 90': 'bpassesrcvd90',
    'Exits per 90': 'exits90',
    'Aerial duels per 90.1': 'gkaerduels90',
    'Free kicks per 90': 'fk90',
    'Direct free kicks per 90': 'directfk90',
    'Direct free kicks on target, %': 'directfkontrgtpct',
    'Corners per 90': 'corners90',
    'Penalties taken': 'pens',
    'Penalty conversion, %': 'pencnvrspct'
}

апл лига1 бундеслига лалига сериаа 23/24->18-19

In [16]:
def parse_names(x, full=False):
        splitter = x.find('.')
        if splitter == -1:
            return x[:len(x)//2]
        elif(x.count('.') > 1):
            x = x.split('.')
            if full:
                return '.'.join(x[len(x)//2:])[:-1]
            else:
                return  '.'.join(x[:len(x)//2])[:-1]
        else:
            if full: 
                return x[splitter-1:]
            else:
                return x[:splitter-1]



# def link_name(market, stats):
#     x = market['short_name'].apply(lambda x: extractOne(x, stats.Player.unique().tolist()))
#     y = market['name'].apply(lambda x: extractOne(x, stats.Player.unique().tolist()))
#     link = []
#     for i in range(len(x)):
#         if x.iloc[i][1] > y.iloc[i][1]:
#             name = x.iloc[i][0]
#             score = x.iloc[i][1]
#         else:
#             name = y.iloc[i][0]
#             score = y.iloc[i][1]

#         if score < 90:
#             link.append((np.nan, score))
#         else:
#             link.append((name, score))
        
#     return pd.Series(link).apply(lambda x: x[0])

def link_name(market, stats):
    x = market['short_name'].apply(lambda x: extractOne(x, stats.Player.unique().tolist()))
    y = market['name'].apply(lambda x: extractOne(x, stats.Player.unique().tolist()))
    link = []
    for i in range(len(x)):
        if x.iloc[i][1] > y.iloc[i][1]:
            name = x.iloc[i][0]
            score = x.iloc[i][1]
        else:
            name = y.iloc[i][0]
            score = y.iloc[i][1]

        if score < 90:
            link.append((np.nan, score))
        else:
            link.append((name, score))
        
    return pd.Series(link).apply(lambda x: x[0])


def get_season(x):
    year_to_season = {
        2018 : '18/19',
        2019 : '19/20',
        2020 : '20/21',
        2021 : '21/22',
        2022 : '22/23',
        2023 : '23/24',
    }
    date_ser = pd.to_datetime(x)
    year = date_ser.year if (date_ser.month >= 8 and date_ser.day >= 1) else date_ser.year - 1
    return year_to_season[year]


id_to_league = {
    'GB1' : 'premier-league',
    'FR1' : 'ligue-1',
    'L1' : 'bundesliga',
    'ES1' : 'laliga',
    'IT1' :'serie-a',
}

league_to_league_name = {
    'premier-league': 'Premier League',
    'ligue-1': 'Ligue 1',
    'bundesliga': 'Bundesliga',
    'laliga': 'LaLiga',
    'serie-a': 'Serie A',
}

In [17]:
df = pd.read_excel(Path.cwd()/f'raw_data/Search results-{1}.xlsx')
df.rename(columns=abbreviations)

,player,teamcurr,team,pos,age,marketval,ctcexp,matchesplayed,minsplayed,goals,...,prevgoals90,bpassesrcvd90,exits90,gkaerduels90,fk90,directfk90,directfkontrgtpct,corners90,pens,pencnvrspct
0,E. Haaland,Manchester City,Manchester City,CF,23,170000000,2027-06-30,15,1448,14,...,NaN,0.06,NaN,NaN,0.06,0.06,0.0,0.00,4,75.0
1,P. Foden,Manchester City,Manchester City,"RAMF, AMF, RW",23,110000000,2027-06-30,20,1780,5,...,NaN,0.20,NaN,NaN,0.35,0.00,0.0,1.01,0,0.0
2,B. Saka,Arsenal,Arsenal,"RWF, RW",22,110000000,2027-06-30,20,1945,6,...,NaN,0.00,NaN,NaN,0.09,0.05,0.0,3.52,2,100.0
3,E. Fernández,Chelsea,Chelsea,"LDMF, LCMF",23,85000000,2031-06-30,18,1531,2,...,NaN,0.41,NaN,NaN,0.41,0.18,0.0,0.94,2,50.0
4,C. Nkunku,Chelsea,Chelsea,"AMF, CF, LCMF",26,80000000,2026-06-30,3,153,1,...,NaN,0.00,NaN,NaN,0.00,0.00,0.0,0.00,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,O. Bobb,Manchester City,Manchester City,"LAMF, LCMF, LW",20,0,2026-06-30,7,152,1,...,NaN,0.00,NaN,NaN,0.00,0.00,0.0,0.00,0,0.0
496,M. Olakigbe,Brentford,Brentford,"RWF, LWF, LAMF",19,0,2026-06-30,8,119,0,...,NaN,0.00,NaN,NaN,0.00,0.00,0.0,0.00,0,0.0
497,Youssef Chermiti,Everton,Everton,CF,19,0,NaN,7,92,0,...,NaN,0.00,NaN,NaN,0.00,0.00,0.0,0.98,0,0.0
498,J. Donley,Tottenham Hotspur U21,Tottenham Hotspur,"CF, DMF",19,0,2027-06-30,2,8,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0


In [18]:
apps = pd.read_csv(Path.cwd()/'archive-4/appearances.csv')
apps = apps[(apps.competition_id.isin(['GB1', 'FR1', 'TR1', 'NL1', 'SC1', 'IT1', 'DK1', 'PO1', 'ES1', 'BE1', 'RU1', 'GR1', 'L1'])) & (apps.date > '2018-08-01')]
apps['season'] = apps['date'].apply(get_season)
apps = apps.groupby(['player_id', 'player_club_id', 'season', 'competition_id']).first().reset_index()[['player_id', 'player_club_id', 'season', 'competition_id']]
apps

,player_id,player_club_id,season,competition_id
0,532,86,18/19,L1
1,532,86,19/20,L1
2,1662,33,18/19,L1
3,1667,162,18/19,FR1
4,2421,38,18/19,L1
...,...,...,...,...
38653,1166093,30120,23/24,GR1
38654,1173042,601,23/24,BE1
38655,1176345,273,23/24,FR1
38656,1176866,3840,23/24,TR1


In [19]:
market = pd.read_csv(Path.cwd()/'archive-4/player_valuations.csv')
players = pd.read_csv(Path.cwd()/'archive-4/players.csv')
comps = pd.read_csv(Path.cwd()/'archive-4/competitions.csv')
teams = pd.read_csv(Path.cwd()/'archive-4/clubs.csv')

market = market[market.date >'2018-08-01']
market['season'] = market['date'].apply(get_season)
market = market.merge(apps, how='left', on=['player_id', 'season'])
market = market.merge(teams[['club_id', 'club_code', 'name']].rename({'name': 'club_name'}, axis=1), how='left', left_on='player_club_id', right_on='club_id')
market = market.groupby(by=['player_id', 'player_club_id', 'season', 'competition_id']).last().reset_index()


players['short_name'] = players['first_name'].str[0].fillna('') + '. ' + players['last_name']


market = market.merge(players[['short_name', 'player_id', 'name']], how='inner', on='player_id').dropna()
market['league'] = market['competition_id']
market = market[['player_id', 'season', 'club_code', 'club_name', 'short_name', 'name', 'league', 'market_value_in_eur']]

In [20]:
def market_value(row, thresh=90):
    if row['player'] == 'A. Gomes': row['player'] = 'Angel Gomes'
    if row['player'] == 'L. Pellegrini': row['player'] = 'Lorenzo Pellegrini'
    if row['player'] == 'S. Dursun': row['player'] = 'Serdar Dursun'
    if row['player'] == 'St. Kelly': row['player'] = 'Stephen Kelly'
    if row['player'] == 'S. Kelly': row['player'] = 'Stephen Kelly'
    if row['player'] == 'Se. Kelly': row['player'] = 'Sean Kelly'
    

    m = market[(market.season == row['season']) & (market.league == row['league'])]
    club = extractOne(row.team, choices=m.club_code.unique())[0]
    m = m[m.club_code == club]
    tm_short = extractOne(query=row['player'], choices=m.short_name, score_cutoff=thresh) or (None, 0)
    tm_long = extractOne(query=row['player'], choices=m['name'], score_cutoff=thresh) or (None, 0)
        
    if tm_short == (None, 0) and tm_long == (None, 0):
        row['playerid'] = np.nan
        row['marketval'] = np.nan
        return row
    else:
        if m[m.short_name == tm_short[0]].shape[0] > 1 or m[m['name'] == tm_long[0]].shape[0] > 1:
            display(row, m[m.short_name == tm_short[0]], m[m['name'] == tm_long[0]])
        if tm_short[1] > tm_long[1]:
            row['playerid'] = m[m.short_name == tm_short[0]]['player_id'].item()
            row['marketval'] = m[m.short_name == tm_short[0]]['market_value_in_eur'].item()
        else:
            row['playerid'] = m[m['name'] == tm_long[0]]['player_id'].item()
            row['marketval'] = m[m['name'] == tm_long[0]]['market_value_in_eur'].item()

    return row


In [21]:
competitions = ['GB1', 'FR1', 'TR1', 'NL1', 'SC1', 'IT1', 'DK1', 'PO1', 'ES1', 'BE1', 'RU1', 'GR1', 'L1']
dfs_leagues = [pd.DataFrame() for _ in range(len(competitions))]
df_all = pd.DataFrame()
for i in tqdm(range(len(competitions))):
    for j in tqdm(range(1, 7)):
        if competitions[i] == 'RU1' and j == 1:
            continue
        df = pd.read_excel(Path.cwd()/f'wyscout/{competitions[i]}/Search results-{j}.xlsx')
        df = df.rename(columns=abbreviations)

        if j == 1:
            season = '23/24'
        elif j == 2:
            season = '22/23'  
        elif j == 3:
            season = '21/22'
        elif j == 4:
            season = '20/21'
        elif j == 5:
            season = '19/20'
        else:
            season = '18/19'

        df['league'] = competitions[i]
        df['season'] = season
        df['team'] = df['team'].apply(
            lambda x: x.replace('Internazionale', 'Inter Milan') \
                            .replace('PSG', 'Paris Saint-Germain') \
                            .replace('Espanyol', 'Espanyol Barcelona')
            )
        df['playerid'] = np.nan
        df = df.apply(market_value, axis=1, result_type='expand')
        df.marketval /= 10**6
        # df['league'] = df['league'].apply(lambda l: league_to_league_name[l])
        df = df.iloc[:, [117, 0, 2, 3, 5, 115, 116]+ [i for i in range(7, 115)]]
        df = df.dropna(subset=['playerid'])
        df = df.set_index(['playerid', 'player', 'league', 'team', 'season'])
        if df.index.duplicated(keep=False).any(): 
            display(dfs_leagues[i][dfs_leagues[i].index.duplicated(keep=False)])
            display(df[df.index.duplicated(keep=False)])
        df_all = pd.concat([df_all, df])

stats = df_all.reset_index()
stats = stats.dropna(subset=['playerid'])


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [22]:
rasie

NameError: name 'rasie' is not defined

In [23]:
stats

,playerid,player,league,team,season,pos,marketval,matchesplayed,minsplayed,goals,...,prevgoals90,bpassesrcvd90,exits90,gkaerduels90,fk90,directfk90,directfkontrgtpct,corners90,pens,pencnvrspct
0,246963.0,C. Morris,GB1,Luton Town,23/24,CF,13.0,36,2987,10,...,NaN,0.03,NaN,NaN,0.00,0.00,0.0,0.00,4,100.000
1,395516.0,M. Diaby,GB1,Aston Villa,23/24,"CF, AMF",60.0,36,2249,6,...,NaN,0.24,NaN,NaN,0.00,0.00,0.0,0.04,0,0.000
2,525247.0,M. Kilman,GB1,Wolverhampton Wanderers,23/24,"RCB, LCB, CB",35.0,36,3687,2,...,NaN,1.15,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000
3,495666.0,W. Saliba,GB1,Arsenal,23/24,RCB,75.0,36,3656,2,...,NaN,1.35,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000
4,258889.0,D. Solanke,GB1,Bournemouth,23/24,CF,25.0,36,3584,18,...,NaN,0.00,NaN,NaN,0.00,0.00,0.0,0.00,3,66.667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31295,31159.0,M. Harnik,L1,Werder Bremen,18/19,"CF, RWF",2.0,18,888,4,...,NaN,0.30,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000
31296,39094.0,S. Langkamp,L1,Werder Bremen,18/19,"RCB, LCB",2.0,21,1501,0,...,NaN,3.36,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000
31297,19819.0,Raffael,L1,Borussia M'gladbach,18/19,CF,2.5,13,556,2,...,NaN,0.00,NaN,NaN,0.32,0.16,0.0,0.65,0,0.000
31298,31041.0,F. Johnson,L1,Borussia M'gladbach,18/19,"RB, RWF, LW",3.0,18,1068,1,...,NaN,1.10,NaN,NaN,0.00,0.00,0.0,0.00,0,0.000


In [ ]:
stats[stats['team'] == 'Paris Saint-Germain']

,playerid,player,league,team,pos_23/24,marketval_23/24,season_23/24,matchesplayed_23/24,minsplayed_23/24,goals_23/24,...,prevgoals90_18/19,bpassesrcvd90_18/19,exits90_18/19,gkaerduels90_18/19,fk90_18/19,directfk90_18/19,directfkontrgtpct_18/19,corners90_18/19,pens_18/19,pencnvrspct_18/19
1434,342229.0,K. Mbappé,FR1,Paris Saint-Germain,"CF, LWF",180000000.0,23/24,28.0,2253.0,26.0,...,NaN,0.00,NaN,NaN,0.04,0.04,0.0,0.22,2.0,50.0
1437,281963.0,L. Hernández,FR1,Paris Saint-Germain,"LB, LCB",45000000.0,23/24,27.0,2066.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1455,487469.0,Vitinha,FR1,Paris Saint-Germain,"LCMF, RCMF, DMF",45000000.0,23/24,27.0,2203.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1475,708265.0,B. Barcola,FR1,Paris Saint-Germain,"LWF, LWB, RAMF",30000000.0,23/24,26.0,1615.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1476,550550.0,Gonçalo Ramos,FR1,Paris Saint-Germain,CF,50000000.0,23/24,26.0,1403.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2706,182913.0,A. Rabiot,FR1,Paris Saint-Germain,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.24,NaN,NaN,0.00,0.00,0.0,0.08,0.0,0.0
2717,371141.0,S. Nsoki,FR1,Paris Saint-Germain,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.85,NaN,NaN,0.00,0.00,0.0,0.00,0.0,0.0
2772,370846.0,T. Weah,FR1,Paris Saint-Germain,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,NaN,NaN,0.00,0.00,0.0,0.00,0.0,0.0
2775,395512.0,A. Bernede,FR1,Paris Saint-Germain,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.51,NaN,NaN,0.00,0.00,0.0,0.00,0.0,0.0


In [24]:
stats.to_csv('prepped/player_stats2.csv', index=False)